In [1]:
import pandas as pd
import numpy as np
import random
import os
from collections import Counter

# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = [
    # 'Oakland Middle School',
    # 'Siegel Middle School',
    # 'Whitworth-Buchanan Middle School',
    # 'Christiana Middle School',
    # 'Smyrna Middle School',
    # 'Stewarts Creek Middle School',
    # 'Rockvale Middle School',
    # 'Rocky Fork Middle School',
    # 'Blackman Middle School',
    # ## 'Thurman Francis Arts Academy',
    # 'Rock Springs Middle School',
    'LaVergne Middle School'
]

# get event year
try:
    event_year = int(input('Enter 4-digit year of YouScience Event\n'))
except ValueError:
    event_year = 2022
print(f"Running for YS Career Fair {event_year}")

# preparing directories TEST needs to be update in fnc export_POS_rosters(), also export_room_assignments()
log_pathway = f'../Logs/{event_year}'
if not os.path.exists(log_pathway):
    os.makedirs(log_pathway)
for school in schools:
    path = f'../YouScienceData/Schedules/{event_year}/{school}'
    pathway_path = f'../YouScienceData/Schedules/{event_year}/{school}/Pathway_Rosters'
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(pathway_path):
        os.makedirs(pathway_path)
    

Running for YS Career Fair 2023


# FUNCTIONS DOCUMENTATION

IN_PROCESS: Updated from Alpha series. Organization has been updated to include {event_year} in file system. 
<table>
    <tr>
        <th>Term</th>
        <th>Definition/Explanation</th>
    </tr>
    <tr>
        <td>POS</td>
        <td>"Pathways of Study", w/o context it is used synomously with pathway(s). These are essentially potential course mappings that students would follow for their highschool credits.</td>
    </tr>
    <tr>
        <td>Cluster</td>
        <td>From YouScience, focus areas matched to students based on their perceived aptitude and interests. Prioritized by RuCo ranking system (see beta_get_criteria.ipynb).</td>
    </tr>
    <tr>
        <td>Sorting Rounds</td>
        <td>Each of the 4 blocks is sorted in two rounds (except the first/fourth) before moving to the next block. 
            <ol>
                <li>In round 1, <i>fill_pos_block()</i> fills pathway rosters by POS demand and returns students who did not get sorted due to no available matches.</li>
                <li>Between rounds, <i>check_for_missed()</i> takes unasigned students from round 1 who DO have YS results and prioritizes them for the next round of sorting after the students have been randomly ordered.</li>
                <li>Then, round 2 of sorting begins with <i>backfill_pos()</i> where unassigned students are randomly assigned to fill the rooms of pathways with remaining open seats evenly.</li>
            </ol>
        </td>
    </tr>
</table>

### Format: <strong>Function_Name</strong> (<i>parameters</i>), description, <i>-f- required files</i>, <i>-o- required objects</i>.</br>

Clarify: -f- required file, -F- required file Folder, -fnc- required function, " " indicates variable name

<ul>
    <li><strong>get_POS_from_clusters</strong>(<i>school</i>), returns school-specific df of rows that contain student info and ranked POS matches or 0's. 0's indicate either school-unsupported YS clusters or students with no YS results. </br></br><i>-f- direct_join_prepared.xlsx, -F- YouScienceData/YS_Criteria_by_School</i></li></br>
    <li><strong>get_POS_demand</strong>(<i>school_pos_df</i>), tabulates and return the demand Counter object (dict-like) of each POS offered at a given school, flag for demand met is deprecated here.</br></br><i>-o- school_pos_df</i></li></br>
    <li><strong>uncouple_POS_matches</strong>(<i>demand_counter_object</i>), CALLED by fnc initialize_block_rosters(), it returns a desc-ordered list of individual school-specific pathways by breaking up grouped pathways from direct_join_prepared.xlsx.</br></br><i>-o- demand Counter object</i></li></br>
    <li><strong>extract_capacity_vector</strong>(<i>school</i>), CALLED by initialize_block_rosters(), reads in and extracts capacity vector object as a dict per the school from capacity_vectors.csv.</br></br><i>-f- Reports/{event_year}/capacity_vectors.csv</li></br>
    <li><strong>initialize_block_rosters</strong>(<i>school, demand_object</i>), demand object should be counter object of school returned by get_POS_demand(), returns structured dictionary for school pathway rosters and a capacity vector dictionary with assigned POS per room.</br></br><i>-fnc- uncouple_POS_matches(), -fnc- extract_capacity_vector(), -o- demand Counter object</i></li></br>
    <li><strong>roster_status</strong>(<i>block='B1'</i>), CALLED by fill_POS_block(), CALLED by backfill_POS(), calls log file object to print the current size of each POS class per specified block and the proportion of 8th graders at the school currently assigned to a POS per the specified block, assumed block 1.</br></br><i>-o- 'logf', -o- 'school', -o- 'school_pos_df', -o- "pos_rosters" (from initialize_block_rosters), -o- block (options: ['B1', 'B2', 'B3', 'B4'] )</i></li></br>
    <li><strong>check_for_missed</strong>(<i>ranked_df, current_block</i>), CALLED by fill_POS_block(), prioritizes students with YS results who did not get their top match within a given block before the assignments of the next block. See table above for more detail. </br></br><i>-o- "missed_took_YS" (collection of students who were missed by the previous block assignments by fill_pos_block() <strong>AND</strong> have YS results) (collection of filtered, returned objects) </i></li></br>
    <li><strong>fill_POS_block</strong>(<i>block='B1'</i>), returns roster of remaining students who have not been assigned a pathway in the given block AND returns open seats vector for backfilling block rosters.</br></br><i>-o- initialized block rosters (ex. 'B1_roster'),-o- "pos_rosters", -o- "school_pos_df", -o- "logf", -o- 'all_capacity_vectors' (dict of outputs of cap vector dict from initialize_block_rosters() ), -o- 'school'</br></br>-fnc- roster_status(), -fnc- check_for_missed()</i></li></br>
    <li><strong>backfill_pos</strong>(<i>block='B1'</i>), evenly distributes unassigned students across POS rosters to open seats during round 2 of assignments within a block. See table above for more detail.</br></br><i>-o- "unassigned" (object returned by fill_POS_block() ), -o- "OS_vector" (object returned by fill_POS_block() ), -o- "pos_rosters"</i></li></br>
    <li><strong>append_schedule</strong>(<i></i>), adds students' schedules for YS event to school_pos_df object for export purposes.</br></br><i>-o- "school_pos_df"</i></li></br>
    <li><strong>clean_to_export</strong>(<i>school_pos_df, school</i>), returns cleaned COMPLETE df ready for export and distribution to schools/county admin. Contains schedules arranged by homeroom teacher and student last name for all students at school including (at time of writing) 5 open schedules to be claimed by last minute new students.</br></br><i>-F- Skyward/{event_year-1}-{event_year}/</i></li></br>
    <li><strong>export_POS_rosters</strong>(<i>pos_keys, cleaned_df</i>), exports individual pathway rosters for each school in the form of an excel workbook with 5 sheets ("All", "B1", ...).</br></br><i>-o-cleaned_df from clean_to_export(), -o- pos_keys (assumed list(pos_rosters.keys()) format)</i></li></br>
    <li><strong>export_room_assignments</strong>(<i>school_cap_vector</i>), exports POS room assignments to appropriate school schedule folder, intended for updating room assignments during schedule making.</br></br><i>-o- capacity vector object returned by initialize_block_rosters(), -o- "event_year", -o- "school", -F- YouScienceData/Schedules/Event-year/school</i></li></br>

</ul>

In [4]:
# functions
def get_POS_from_clusters(school):
    # read in YouScience Ranked criteria file
    ys_path = f'../YouScienceData/YS_Criteria_by_School/{event_year}/{school} YSCriteria.csv'
    ys_match_df = pd.read_csv(ys_path)
    # Read in appropriate translation columns form direct_join_prepared.xlsx
    dj_path = f'../direct_join_prepared.xlsx'
    djp_df = pd.read_excel(dj_path)
    djp_df = djp_df[['YouScience Clusters',school]]

    # create replacement dictionary
    to_replace = {}
    for i in range(len(djp_df)):
        cluster = djp_df.iloc[i]
        # key/value = YS cluster/school's coresponding POS
        if type(cluster[school]) != float:
            to_replace[cluster['YouScience Clusters']] = cluster[school]
        else:
            to_replace[cluster['YouScience Clusters']] = 0
    
    # at some point '0' is introduced somewhere. I suspect from the added positions in updated rosters
    to_replace['0'] = 0

    return ys_match_df.replace(to_replace=to_replace).drop('Unnamed: 0', axis=1)

def get_POS_demand(school_pos_df):
    # previously, returned the number of lg rooms needed to calculate capacity vectors later
    # capacity vectors are now generated by beta_capacity_report.py
    C = Counter()
    for i in range(len(school_pos_df)):
        # read in student's POS
        student = school_pos_df.iloc[i]
        # compile list of POS
        pos_list = []
        for rank in ['First','Second','Third','Fourth','Fifth','Sixth']:
            if student[rank] not in pos_list:
                pos_list.append(student[rank])
        # update counter
        # remove redundancy of double matches
        pos_list = list(set(pos_list))
        C.update(pos_list)

    return C

def uncouple_POS_matches(demand_counter_object):
    # Non-Empty matches only
    NE_pos_list = [] 
    for pos_group in demand_counter_object.most_common():
        # passing 0, which would be number of ranked choices that didn't translate to school-offered pathway
        if pos_group[0] == 0:
            continue
        
        # Break up the Coupled POS groups into separate, equally-demanded pathways
        if ',' in pos_group[0]:
            # temp list of pathways
            l = pos_group[0].split(', ')
            for pos in l:
                NE_pos_list.append([pos, pos_group[1]])
        # If not coupled, pass in as is 
        else:
            NE_pos_list.append(list(pos_group))

    # check and consolidate many-clusters-to-one POS matching
    if len(set([x[0] for x in NE_pos_list])) == len(NE_pos_list):
        return NE_pos_list

    test_list2 = []
    repeated_list = [y[0] for y in Counter([x[0] for x in NE_pos_list]).most_common() if y[1]>1]
    for rep_pos in repeated_list:
        n = 0
        for pos_couplet in NE_pos_list:
            if pos_couplet[0] == rep_pos:
                n+= pos_couplet[1]
            elif (pos_couplet[0] not in repeated_list) and (pos_couplet not in test_list2):
                test_list2.append(pos_couplet)
        test_list2.append([rep_pos, n])

    # add in non-repeated pos couplets
    test_list2.sort(key=lambda x: x[1], reverse=True)

    return test_list2

def extract_capacity_vector(school):
    # read in capacity vector object 
    df = pd.read_csv(f'../YouScienceData/Reports/{event_year}/capacity_vectors.csv')
    df.drop("Unnamed: 0", axis=1, inplace=True)

    # inital object is long str
    vector_str = df.loc[df.School == school, "Capacity_Vector"].values[0]
    if vector_str == '{}':
        print(f'{school} missing data.')
        return 0
    # transforms it into list of room: cap strings
    vector_list = vector_str.split(', ')

    # transforming into actionable dictionary vector
    vector_dict = {}
    for item in vector_list:
        k, v = item.split(': ')
        # checking for first item to have '{'
        if '{' in k:
            k = k[1:]
        # checking for last item to have '}' 
        if '}' in v:
            v = v[:-1]
        try:
            vector_dict[int(k)] = int(v)
        except ValueError:
            # shave off quotations 
            vector_dict[k[1:-1]] = int(v)
    
    return {k: v for k, v in sorted(vector_dict.items(), key=lambda item: item[1], reverse=True)}

def initialize_block_rosters(school, demand_object):
    # initialize dictionary for POS rosters per school
    pos_rosters = {}
    
    # extract capacity vector dictionary for school
    cap_vector_dict = extract_capacity_vector(school)
    # list of rooms (cap_vector_dict.keys()), in DESC order of capacity
    rooms = list(cap_vector_dict.keys())
    assigned_cap_vector_dict = {}
    i = 0
    for pos_couplet in uncouple_POS_matches(demand_object):
        # pos_key will be a doubleton set [pos_key, demand value] in DESC order
        pos_rosters[pos_couplet[0]] = {
            'All':[],
            'B1':[],
            'B2':[],
            'B3':[],
            'B4':[],
        }

        # restructuring cap vector {room:int(cap)} -> {room: arr[int(cap), str(POS assignment)]}
        try:
            assigned_cap_vector_dict[rooms[i]] = [cap_vector_dict[rooms[i]], pos_couplet[0]]
        except IndexError:
            print(f'{school} provided {i} rooms for {len(uncouple_POS_matches(demand_object))} pathways') 
            break
        i+=1

    return pos_rosters, assigned_cap_vector_dict

def roster_status(block='B1'):
    x = 0
    # calls pos_rosters because I want it changed rather than a local version
    for pos in list(pos_rosters.keys()):
        x += len(pos_rosters[pos][block])
        logf.write(f'{block}_Status: {school}-{pos}, {len(pos_rosters[pos][block])}\n')

    logf.write(f'{block}_Fill_Status: {school} {x}/{len(school_pos_df)}\n')
    return

def check_for_missed(ranked_df, current_block):
    # looking at previous block
    prev_block = {
        'B2':'B1',
        'B3':'B2',
        'B4':'B3',
    }
    # will be unneccessary for first block assignments
    if current_block == 'B1':
        return ranked_df
    else:
        block = prev_block[current_block]

    # current list of students with POS match   
    student_ids = list(ranked_df.id)

    # initializing lcoations of priority students with POS match
    target_indices = []

    # checking for priority students
    for id in missed_took_YS[school][block]:
        if id in student_ids:
            ind = ranked_df.loc[ranked_df.id == id].index.values[0]
            target_indices.append(ind)
    
    # preparing new_index to reindex ranked_df before continuing
    new_index = [x for x in target_indices]
    # adding back existing student_ids
    new_index += [i for i in range(len(ranked_df)) if i not in new_index]

    # reindex ranked_df with priority students in front and index reset to start w/ 0
    return ranked_df.reindex(labels=new_index, axis='index').reset_index(drop=True)

def fill_POS_block(block='B1'):
    # initialize reference block roster
    remaining_roster = eval(f'{block}_roster')
    pos_list = list(pos_rosters.keys())
    # creating open_seats_vector
    open_seats_vector = []
    # iterating over POS's in block until all students assigned pos during block
    for pos_key in pos_list:
        # initialize choice to filter by
        rankings = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth']
        r = 0
        rank = rankings[r]

        # initialize randomized slice of pos_df and index counter, check for priority students
        ranked = school_pos_df.loc[school_pos_df[rank] != 0].sample(frac=1).reset_index(drop=True)
        ranked = check_for_missed(ranked_df=ranked,current_block=block)
        i = 0

        # initialize size of course
        n = 0
        cap = list(all_capacity_vectors[school].values())[pos_list.index(pos_key)][0]


        # filling
        while n < cap:
            try:
                student, choice = ranked.iloc[i].id, ranked.iloc[i][rank]
            except IndexError:
                logf.write(f'fill_status: {school}, {n} of {cap} students assigned to {pos_key}, {rank} choice demand met\n')
                # move to next choice ranking
                r += 1
                # check if all rankings have been examined
                if r > 5:
                    logf.write(f'fill_status: {school}, {n} students assigned to {pos_key}, All choices examined.\n')
                    # check for remaining seats 
                    open_seats = cap - n
                    break
                # else update the df slice for consideration
                rank = rankings[r]
                ranked = school_pos_df.loc[school_pos_df[rank] != 0].sample(frac=1).reset_index(drop=True)
                ranked = check_for_missed(ranked_df=ranked,current_block=block)
                # reset index counter
                i = 0
                continue
            # NOTE: criteria for assigning student to pos: 
            #   ->(i) matched, 
            #   ->(ii) isn't already matched for current block, 
            #   ->(iii) isn't already matched for this pos prior
            if (pos_key in choice) and (student in remaining_roster) and (student not in pos_rosters[pos_key]['All']):
                # assign to master
                pos_rosters[pos_key]['All'].append(student)
                # assign to block specific
                pos_rosters[pos_key][block].append(student)
                n += 1
                remaining_roster.remove(student)
            i += 1
            if n == cap:
                logf.write(f'fill_status: {school}, {pos_key} {block} filled.\n')
                # hence no remaining open seats in POS
                open_seats = 0

        open_seats_vector.append(open_seats)

    # checking status
    roster_status(block=block)

    return remaining_roster, open_seats_vector

def backfill_POS(block='B1'):
    # initialize randomized remaining list
    r_unassigned = random.sample(unassigned, len(unassigned))

    while len(r_unassigned) > 0:
        # index of first unassigned student 
        check_ind = 0

        # get index of most empty POS
        ind = OS_vector.index(max(OS_vector))
        pos_key = list(pos_rosters.keys())[ind]

        try:
            # if student is already assigned to pathway, find first who isn't 
            while r_unassigned[check_ind] in pos_rosters[pos_key]['All']:
                check_ind += 1

            # adding student to POS roster
            pos_rosters[pos_key]['All'].append(r_unassigned[check_ind])
            pos_rosters[pos_key][block].append(r_unassigned[check_ind])
            # noting student added to that POS
            OS_vector[ind] -= 1
            # removing student from unassigned list
            r_unassigned.pop(check_ind)

        except IndexError: # last student already assigned to that pathway (e.g. "pos_key") 
            # can't continue, because it will keep trying that first/most-open pos_key, so force key change
            OS_vector[ind] -= 1
        

    # tracking progress
    roster_status(block=block)
    return len(r_unassigned)

def append_schedule(): #designed with the idea of a class object method in mind
    # create empty columns for export
    for block in ['B1','B2','B3','B4', 'CY Teacher']:
        school_pos_df[block] = ''

    # iterate through POS offerings to fill in schedule B1-B4
    for pos_key in list(pos_rosters.keys()):
        # iterate through blocks
        for block in ['B1','B2','B3','B4']:
            # iterate through Block roster
            for id in pos_rosters[pos_key][block]:
                # # find and write pos under student's corresponding slot
                # row = school_pos_df.loc[school_pos_df.id == id].index.values
                # school_pos_df.loc[school_pos_df.index[row], block] = pos_key
                # # pretty sure this should work
                school_pos_df.loc[school_pos_df.id == id, block] = pos_key
    return

def clean_to_export(school_pos_df, school):
    # desired export features
    features = [
        'id',
        'School',
        'First_Name',
        'Last_Name',
        'Email',
        'B1',
        'B2',
        'B3',
        'B4',
        'CY Teacher',
    ]
    
    # Get homeroom/1st period teacher
    school_skyward_path = f'../YouScienceData/Skyward/{event_year - 1}-{event_year}/Skyward_{school}.xlsx'
    school_skyward_df = pd.read_excel(school_skyward_path)

    for email in list(school_pos_df.Email):
        if 'Open' in email: # extra schedules have emails labeled "Open {#}"
            continue
        
        X = school_skyward_df.loc[school_skyward_df['Student\'s School Email'] == email]
        try:
            teacher_last, teacher_first = X['CY Teacher Last Name'].values[0], X['CY Teacher First Name'].values[0]
            school_pos_df.loc[school_pos_df.Email == email, 'CY Teacher'] = str(teacher_last) + ', ' + str(teacher_first)[0]
        except IndexError: # student's info was at a different school
            school_pos_df.loc[school_pos_df.Email == email, 'CY Teacher'] = '0, Missing'
    # alphabetize students & homeroom teacher
    school_pos_df.sort_values(['CY Teacher', 'Last_Name'], inplace=True)

    # space here if we wish to transform further during iteration
    return school_pos_df[features].reset_index(drop=True)

pos_str_exceptions = set()

def export_POS_rosters(pos_keys, cleaned_df):

    for pos_key in pos_keys:
        # fixing unexpected '/' in pos_key str
        new_key = pos_key.replace('/', ', ')
            
        if new_key != pos_key:
            pos_str_exceptions.add(pos_key)
        
        # get block slices, index starting with 1 (teacher-friendly)
        b1_df = cleaned_df.loc[cleaned_df.B1 == pos_key]
        b1_df.index = np.arange(1, len(b1_df) + 1)
        b2_df = cleaned_df.loc[cleaned_df.B2 == pos_key]
        b2_df.index = np.arange(1, len(b2_df) + 1)
        b3_df = cleaned_df.loc[cleaned_df.B3 == pos_key]
        b3_df.index = np.arange(1, len(b3_df) + 1)
        b4_df = cleaned_df.loc[cleaned_df.B4 == pos_key]
        b4_df.index = np.arange(1, len(b4_df) + 1)

        # create .xlsx file for pathway
        with pd.ExcelWriter(f'../YouScienceData/Schedules/{event_year}/{school}/Pathway_Rosters/{new_key}.xlsx') as exwriter:
            b1_df.to_excel(exwriter, sheet_name='Block 1')
            b2_df.to_excel(exwriter, sheet_name='Block 2')
            b3_df.to_excel(exwriter, sheet_name='Block 3')
            b4_df.to_excel(exwriter, sheet_name='Block 4')
    return 

def export_room_assignments(school_cap_vector):
    df = pd.DataFrame(school_cap_vector, columns=school_cap_vector.keys()).T.reset_index()
    df.rename({'index':'Room',0:'Capacity',1:'POS'},axis=1, inplace=True)
    df['School'] = school
    # export path
    p = f'../YouScienceData/Schedules/{event_year}/{school}/room_assignments_by_demand.csv'
    df.to_csv(p)
    return 


In [5]:
log_name = input("Input file_name for log file")
with open(f'../Logs/{event_year}/{log_name}.txt','w') as logf:
    
    # dictionary for school demand for later analysis...
    school_demand = {}

    # tracking students who were unassigned during each block but who took YS test
    missed_took_YS = {}

    # dictionary for school capacity vectors
    all_capacity_vectors = {}

    # begin
    for school in schools:
        
        # initialize school_pos_df with formatted YS results
        school_pos_df = get_POS_from_clusters(school=school)

        # initialize full block rosters
        B1_roster = school_pos_df.id.to_list()
        B2_roster = school_pos_df.id.to_list()
        B3_roster = school_pos_df.id.to_list()
        B4_roster = school_pos_df.id.to_list()

        # initialize missed_took_YS for school
        missed_took_YS[school] = {
            'B1':[],
            'B2':[],
            'B3':[],
            'B4':[],
        }

        # get POS demand Counter object
        school_demand[school] = get_POS_demand(school_pos_df)

        # initialize pathway rosters and extract school capacity vector
        pos_rosters, all_capacity_vectors[school] = initialize_block_rosters(school=school, demand_object=school_demand[school])

        # begin rounds for assignments
        for block in ['B1', 'B2','B3','B4']:
            
            # Round 1, see table above for details
            unassigned, OS_vector = fill_POS_block(block=block)

            # Prepare for next block, filter students with YS results
            missed_took_YS[school][block] = [x for x in unassigned if ('Missing' not in x) and ('Empty' not in x)]

            # Round 2, see table above for details
            backfill_POS(block=block)

        # attach schedules to school_pos_df 
        append_schedule()
        
        # clean complete schedule for export
        cleaned_df = clean_to_export(school_pos_df, school)

        # export complete schedule table
        clean_p = f'../YouScienceData/Schedules/{event_year}/{school}/{school}_all_students.csv'
        cleaned_df.to_csv(clean_p)

        # export POS rosters
        export_POS_rosters(cleaned_df=cleaned_df, pos_keys=list(pos_rosters.keys()))

        # export Room assignments
        export_room_assignments(all_capacity_vectors[school])

logf.close()

# Thoughts and Development

## get_POS_demand()
<ul>
    <li><strong>get_POS_demand()</strong>: previously it returned the number of lg rooms for the purpose of creating capacity vectors</li></br>
    <li>Curently, I'm thinking read in the capacity report outside of the iterative loop, and read in/extract the relevant capacity vector per school.</li></br>
    <li>I think the question is whether or not identifying if demand per POS could be met is a value-added for now? I think it is, and the move is to read in the appropriate vector here and compare.</li>
</ul>

### Decision

This is inappropriate to do so here before the uncoupling of bundled POS. Otherwise, such assignments or checks is premature and misleading. For example in C.most_common()[1] for schools[0] is a 3-bundled POS. Comparing len(rooms) to len(C.most_common()) results in 19 to 13, but there are actually 19 POS pathways. It's not 6 rooms extra. It's at minimum viable.</br></br> <strong><u>!!!FLAG!!!</u></strong></br>

### Future Development

Possibility: add in clause to let the algorithm be greedy if there are extra rooms provided by school, based on the capacity report. 

## Capacity Vector structure
<ul>
    <li>Previously, the capacity vector was a simple array of the length of the school's provided rooms with entries of 50/35 per lg/sm room distinction.</li></br>
    <li>In an attempt to handle both the much broader range of capacities per room and make assigning rooms easier in the scheduling process, a school's capacity vector as built by beta_capacity_report.py is a dictionary that has room names/numbers as keys and corresponding capacity as values. </li></br>
    <li><strong>IDEA:</strong> add the assigned pathway to the respective key/value pair. For ex: {'Library':40} -> {'Library':[40, 'BioSTEM']}</li></br>
</ul>

### Decision -Delayed

CONCERN: we do not have enough information for this year's event currently. Going off of last year's data we can decide what room assignments could be changed, but this year's data has not included assigned pathways to rooms. 
As is, the assigned capacity vector(s) can be exported as a DF ready to be read in by the schedule building script, and included in schools' folders for organizational purposes. 